In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [7]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
team_names = [team['full_name'] for team in nba_teams]
team_names.sort()
team_ids = [team['id'] for team in nba_teams]

In [8]:
from nba_api.stats.endpoints import leaguegamefinder
games = None
for ids in team_ids:
    if games is None:
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=ids)
        games = gamefinder.get_data_frames()[0]
    else:
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=ids)
        games = pd.concat([games, gamefinder.get_data_frames()[0]])

In [9]:
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])

In [13]:
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612737,ATL,Atlanta Hawks,0022200974,2023-03-06,ATL @ MIA,None,84,56,...,0.500,4.0,8.0,12.0,14,2.0,0,1,9,13.6
1,22022,1610612737,ATL,Atlanta Hawks,0022200958,2023-03-04,ATL @ MIA,L,239,109,...,0.867,10.0,27.0,37.0,22,12.0,1,16,23,-8.0
2,22022,1610612737,ATL,Atlanta Hawks,0022200947,2023-03-03,ATL vs. POR,W,239,129,...,0.813,10.0,33.0,43.0,30,10.0,5,17,19,18.0
3,22022,1610612737,ATL,Atlanta Hawks,0022200923,2023-02-28,ATL vs. WAS,L,239,116,...,0.895,13.0,29.0,42.0,21,4.0,7,9,17,-3.0
4,22022,1610612737,ATL,Atlanta Hawks,0022200911,2023-02-26,ATL vs. BKN,W,240,129,...,0.821,13.0,33.0,46.0,20,7.0,2,15,18,2.0


In [10]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102159 entries, 0 to 2862
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   SEASON_ID          102159 non-null  object        
 1   TEAM_ID            102159 non-null  int64         
 2   TEAM_ABBREVIATION  102159 non-null  object        
 3   TEAM_NAME          102159 non-null  object        
 4   GAME_ID            102159 non-null  object        
 5   GAME_DATE          102159 non-null  datetime64[ns]
 6   MATCHUP            102159 non-null  object        
 7   WL                 102135 non-null  object        
 8   MIN                102159 non-null  int64         
 9   PTS                102159 non-null  int64         
 10  FGM                102159 non-null  int64         
 11  FGA                102159 non-null  int64         
 12  FG_PCT             102152 non-null  float64       
 13  FG3M               102159 non-null  int64     

In [11]:
games['PLUS_MINUS']

0       13.6
1       -8.0
2       18.0
3       -3.0
4        2.0
        ... 
2858     NaN
2859     NaN
2860     NaN
2861     NaN
2862     NaN
Name: PLUS_MINUS, Length: 102159, dtype: float64

In [12]:
games['GAME_DATE'] > "2015-09-01"

0        True
1        True
2        True
3        True
4        True
        ...  
2858    False
2859    False
2860    False
2861    False
2862    False
Name: GAME_DATE, Length: 102159, dtype: bool

In [133]:
games = games[games['GAME_DATE'] > "2022-09-01"].sort_values(by='GAME_DATE', ascending=False)

In [134]:
game_ids = games['GAME_ID'].unique().tolist()

In [135]:
games['TEAM_NAME'].unique()

array(['Phoenix Suns', 'Brooklyn Nets', 'Houston Rockets',
       'Oklahoma City Thunder', 'Charlotte Hornets', 'Chicago Bulls',
       'San Antonio Spurs', 'Dallas Mavericks', 'Golden State Warriors',
       'Utah Jazz', 'Portland Trail Blazers', 'Boston Celtics',
       'LA Clippers', 'Los Angeles Lakers', 'Indiana Pacers',
       'Orlando Magic', 'Memphis Grizzlies', 'Washington Wizards',
       'New York Knicks', 'Milwaukee Bucks', 'Philadelphia 76ers',
       'Sacramento Kings', 'Minnesota Timberwolves', 'Atlanta Hawks',
       'Detroit Pistons', 'Cleveland Cavaliers', 'Toronto Raptors',
       'Miami Heat', 'Denver Nuggets', 'New Orleans Pelicans'],
      dtype=object)

In [136]:
from nba_api.stats.endpoints import boxscoreadvancedv2

In [137]:
boxscores_advanced = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_ids[0])

The command above/below gets the advanced boxscore from a given game id

boxscores_advanced.get_data_frames()[0] returns the advanced boxscore broken up per player

boxscores_advanced.get_data_frames()[1] returns the advanced boxscore broken up per team

In [138]:
boxscores_advanced.get_data_frames()[1]

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022200962,1610612756,Suns,PHX,Phoenix,240:00,130.0,134.0,127.6,128.6,...,9.3,0.618,0.650,1.0,0.200,99.36,97.5,81.25,97,0.539
1,0022200962,1610612742,Mavericks,DAL,Dallas,240:00,127.6,128.6,130.0,134.0,...,10.2,0.591,0.672,1.0,0.197,99.36,97.5,81.25,98,0.461


In [139]:
boxscores_advanced = None
for game_id in game_ids:
    if boxscores_advanced is None:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced = gamefinder.get_data_frames()[1]
    else:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced = pd.concat([boxscores_advanced, gamefinder.get_data_frames()[1]])

In [142]:
boxscores_advanced

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022200962,1610612756,Suns,PHX,Phoenix,240:00,130.0,134.0,127.6,128.6,...,9.3,0.618,0.650,1.0,0.200,99.36,97.5,81.25,97,0.539
1,0022200962,1610612742,Mavericks,DAL,Dallas,240:00,127.6,128.6,130.0,134.0,...,10.2,0.591,0.672,1.0,0.197,99.36,97.5,81.25,98,0.461
0,0022200965,1610612766,Hornets,CHA,Charlotte,240:00,86.3,85.1,100.3,101.0,...,9.9,0.420,0.455,1.0,0.199,100.64,101.0,84.17,101,0.396
1,0022200965,1610612751,Nets,BKN,Brooklyn,240:00,100.3,101.0,86.3,85.1,...,7.9,0.457,0.502,1.0,0.193,100.64,101.0,84.17,101,0.604
0,0022200967,1610612759,Spurs,SAS,San Antonio,240:00,105.5,106.8,136.9,136.5,...,14.6,0.522,0.538,1.0,0.196,104.00,103.5,86.25,103,0.328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0012200003,1610612763,Grizzlies,MEM,Memphis,240:00,105.1,105.9,100.8,100.0,...,21.8,0.558,0.596,1.0,0.197,101.50,101.5,84.58,101,0.576
0,0012200002,50009,Maccabi Ra'anana,MRA,Ra'anana,240:00,78.9,77.1,115.3,117.5,...,16.2,0.347,0.411,1.0,0.197,103.78,104.0,86.67,105,0.204
1,0012200002,1610612746,Clippers,LAC,LA,240:00,115.3,117.5,78.9,77.1,...,23.3,0.591,0.631,1.0,0.198,103.78,104.0,86.67,103,0.796
0,0012200001,1610612764,Wizards,WAS,Washington,240:00,83.5,82.9,92.7,90.6,...,13.3,0.405,0.448,1.0,0.200,103.90,105.5,87.92,105,0.441


In [143]:
features = ['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY', 
       'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
       'EFG_PCT', 'TS_PCT', 'E_USG_PCT', 'E_PACE', 'PACE',
       'PACE_PER40', 'POSS', 'PIE']

In [ ]:
from nba_api.stats.endpoints import boxscoreplayertrackv2

In [ ]:
boxscores = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=game_ids[0])

In [ ]:
boxscores.get_data_frames()[0]

In [72]:
boxscores_advanced = None
for game_id in game_ids:
    if boxscores_advanced is None:
        gamefinder = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=game_id)
        boxscores_advanced = gamefinder.get_data_frames()[0]
    else:
        gamefinder = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=game_id)
        boxscores_advanced = pd.concat([boxscores_advanced, gamefinder.get_data_frames()[0]])

In [74]:
boxscores_advanced

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,SPD,...,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT
0,0022200958,1610612737,ATL,Atlanta,1629631,De'Andre Hunter,F,,30:18,4.00,...,2,3,0.667,4,7,0.571,0.599,1,1,1.000
1,0022200958,1610612737,ATL,Atlanta,1628381,John Collins,F,,16:13,4.15,...,0,1,0.000,1,3,0.333,0.250,0,2,0.000
2,0022200958,1610612737,ATL,Atlanta,203991,Clint Capela,C,,24:56,3.95,...,7,9,0.778,1,1,1.000,0.800,2,7,0.286
3,0022200958,1610612737,ATL,Atlanta,1627749,Dejounte Murray,G,,36:02,4.14,...,2,6,0.333,1,8,0.125,0.213,3,3,1.000
4,0022200958,1610612737,ATL,Atlanta,1629027,Trae Young,G,,32:17,3.92,...,1,4,0.250,1,9,0.111,0.153,0,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,0022100968,1610612765,DET,Detroit,203482,Kelly Olynyk,,,17:29,3.99,...,3,4,0.750,0,0,0.000,0.750,1,1,1.000
22,0022100968,1610612765,DET,Detroit,203585,Rodney McGruder,,,13:48,4.28,...,0,1,0.000,0,2,0.000,0.000,1,1,1.000
23,0022100968,1610612765,DET,Detroit,1630587,Isaiah Livers,,,24:33,4.16,...,0,0,0.000,3,6,0.500,0.500,1,1,1.000
24,0022100968,1610612765,DET,Detroit,1628977,Hamidou Diallo,,DNP - Coach's Decision,0:00,0.00,...,0,0,0.000,0,0,0.000,0.000,0,0,0.000
